In [1]:
import pandas as pd
import os
import cv2
import numpy as np
import json

In [2]:
#Test Code| Version 2#
Year='1934'
Showa='9'
#Read Motherdata
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\"+Year+"\\DataFrame.json"
with open(path, 'r') as j:
     dta = json.loads(j.read())

In [3]:
#Test code| Version 2#
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
os.chdir(path)
df = pd.read_csv(r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Processed_Data/Index/S'+Showa+'.csv')
df=df.drop(df.columns[0], axis=1)

PositionList=['Manager','Leader','Admin','Outsource','Engineer1','Engineer2']

In [8]:
# Function for returning x coordinate of Position name
def Detect_Position(Json,Position):
    NewList=Json['fields']
    Dict=list()
    for d in NewList:
        try:
            newDict={}
            newDict['inferText']=d['inferText']
            newDict['boundingPoly']=d['boundingPoly']
            Dict.append(newDict)
        except KeyError:
            continue
            
    ### Get Coordinates ###
    if Position=='Manager':        
        res = [d
           for d in Dict 
           if ('主事' in d['inferText']) or ('主' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
    
    if Position=='Leader':        
        res = [d
           for d in Dict 
           if ('課長' in d['inferText']) or ('課' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
        
    if Position=='Engineer1':        
        res = [d
           for d in Dict 
           if ('技師' in d['inferText']) or ('師' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
    
    if Position=='Engineer2':        
        res = [d
           for d in Dict 
           if ('技手' in d['inferText']) or ('手' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
        
    if Position=='Admin':        
        res = [d
           for d in Dict 
           if ('事務員' in d['inferText'])]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
        
    if Position=='Outsource':
        res = [d
           for d in Dict 
           if '嘱託員' in d['inferText']]
        if len(res)!=0:
            res = res[0]['boundingPoly']['vertices']
            Edge=max(int(d['x']) for d in res)
            return(Edge)
        else:
            return([])
        
    if Position=='Clerk':
        res = [d
           for d in Dict 
           if d['inferText']=='書記']
        if len(res)!=0:
            res = res[0]['boundingPoly']
            Edge = max(res, key = lambda i : i[0])[0]
            return(Edge)
        else:
            return([])
        
    if Position=='Worker':
        res = [d
           for d in Dict 
           if d['inferText']=='雇']
        if len(res)!=0:
            res = res[0]['boundingPoly']
            Edge = max(res, key = lambda i : i[0])[0]
            return(Edge)
        else:
            return([])
        
### CLOVA FUNCTION ###
import requests
import uuid
import time
import json
import cv2
import base64

api_url = 'https://deelieyxuc.apigw.ntruss.com/custom/v1/1972/ebd01bcbf693d069817622e9839e20914143c7d0d8953eddee40e8b0af96c95b/general'
secret_key = 'S1NmVXpYZlJ0cGJ0ZEFRZXVlbkRkaHFReE9FcHNTQ0U='

def Clova(Dept,Office,ImagePage):
    path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Office_Level\\"+Year+"\\"+Dept+"\\"+Office+"\\"
    with open(path+"Image"+str(ImagePage)+".png",'rb') as f:
         file_data = f.read()

    request_json = {
            'images': [
                {
                    'format': 'jpg',
                    'name': 'demo',
                    'data':base64.b64encode(file_data).decode()}],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000)),
            'lang':'ja'
            }
    payload = json.dumps(request_json).encode("UTF-8")
    headers = {'X-OCR-SECRET': secret_key,
              'Content-Type': 'application/json'}
    response = requests.request("POST", api_url, headers=headers, data = payload)
    Json=json.loads(response.text)['images'][0]
    
    return Json    


class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


In [9]:
#Test code| Version 2#
for n in range(0,len(df)):
    #Show Working office#
    #Extract key info of office
    if n==len(df):
        print('FINISHED!')
        break
    Row  = df.iloc[n]
    NxRow= df.iloc[n+1]
    

    ###Collect Location information###
    Page=int(Row["Page"])
    NxPage=int(NxRow["Page"])

    Dept=Row["Dept"]
    NxDept=NxRow["Dept"]

    Office=Row["Office"]
    NxOffice=NxRow["Office"]
    
    print(Dept,Office)

    #Convert to json via CLOVA
    JsonList=[]
    for l in range(1,len(dta[str(Year)][Dept][Office]['Page_Range'])+1):
        try:
            Json=Clova(Dept,Office,l)                
        except FileNotFoundError:
            break
        JsonList.append(Json)
        Index=0
        for element in JsonList:
            if 'fields' not in list(element.keys()):
                JsonList=JsonList[-0]
            Index=Index+1
        
    #Find Starting X coordinates of Positions.
    PositionListNow=PositionList.copy()
    path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
    Page_Range=dta[str(Year)][Dept][Office]['Page_Range']
    if Page_Range=='NA':
        continue
    CordDict={}
    Page=Page_Range[0]

    if len(JsonList)==7:
        continue
    else:
        for k in range(0,len(JsonList)):
            CordList=list(map(Detect_Position,[JsonList[k]]*len(PositionListNow),PositionListNow))
            Positions= {PositionListNow[i]:CordList[i] for i in range(len(PositionListNow))}
            Positions = {k:v for k,v in Positions.items() if v}    
            print(Positions)

            PosiList = list(Positions.keys())
            CordList = list(Positions.values())

            CordDictNow= {PosiList[i]: {'XLocation':CordList[i],'Page':Page} for i in range(len(PosiList))}
            CordDictNow = {k:v for k,v in CordDictNow.items() if v}
            CordDict.update(CordDictNow)
            Page=Page+1

            for x in list(CordDictNow.keys()):
              PositionListNow.remove(x)

            if len(PositionListNow)==0:
                continue
        dta[Year][Dept][Office]['Position']=CordDict

        ##Add Ending information
        Frame=pd.DataFrame.from_dict(dta[Year][Dept][Office]['Position'],orient='index')
        if len(Frame)==0:
            continue
        Frame=Frame.sort_values(by = ['Page', 'XLocation'], ascending = [True, False])
        if len(dta[Year][Dept][Office]['Position'])==0:
            continue
        Frame["EndLocation"]=list(Frame['XLocation'].shift(-1))[:-1]+list([dta[Year][Dept][Office]["Office_X2"]])
        Frame["EndPage"]=list(Frame['Page'].shift(-1))[:-1]+list([dta[Year][NxDept][NxOffice]["Starting_Page"]])
        display(Frame)
        Frame=Frame.to_dict('index')
        dta[Year][Dept][Office]['Position']=Frame

Admin 秘書課
{'Manager': 100, 'Leader': 136, 'Admin': 51}


,XLocation,Page,EndLocation,EndPage
Leader,136,2,100.0,2.0
Manager,100,2,51.0,2.0
Admin,51,2,189.0,2.0


Admin 職員課
{'Manager': 150, 'Leader': 179}
{'Admin': 118}


,XLocation,Page,EndLocation,EndPage
Leader,179,2,150.0,2.0
Manager,150,2,118.0,3.0
Admin,118,3,314.0,3.0


Admin 文書課
{'Manager': 274, 'Leader': 242}
{'Admin': 469}
{}


...

,XLocation,Page,EndLocation,EndPage
Leader,279,46,258.0,46.0
Manager,258,46,448.0,47.0
Engineer1,448,47,417.0,47.0
Admin,417,47,199.0,47.0
Engineer2,199,47,282.0,50.0


保健局 庶務課
{'Manager': 248, 'Leader': 272, 'Admin': 79}
{'Engineer2': 13}


,XLocation,Page,EndLocation,EndPage
Leader,272,50,248.0,50.0
Manager,248,50,79.0,50.0
Admin,79,50,13.0,51.0
Engineer2,13,51,417.0,51.0


保健局 衛生課


KeyError: 'Page_Range'

In [10]:
json_object = json.dumps(dta, indent=4,
                        cls=NpEncoder)
save_path='C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\'+str(Year)+'\\'
with open(save_path+"DataFrame.json", "w") as outfile:
    outfile.write(json_object)